## Repeat with last year's data to compare with latest year

### Load data for last year

In [ ]:
import pandas as pd

# We use Jan 2020 data
trade_lastyr = pd.read_excel("export_2020m1.xlsx", skiprows=1, index_col=0, nrows=7)
trade_lastyr

### Create network

In [ ]:
# We first convert the matrix data into pair data
trade_lastyr = pd.melt(trade_lastyr.reset_index(), id_vars='index')
trade_lastyr.columns = ['export_country', 'import_country', 'value']
trade_lastyr.head()

In [ ]:
trade_lastyr.dropna(inplace=True)
trade_lastyr.head()

In [ ]:
# Now we can use the pairs to create the graph:
#   (i) first two columns representing the nodes; 
#   (ii) last column representing the weight
# We also set the edge color based on 'export_country'

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

G_lastyear = nx.DiGraph()  #create a directed graph

for index, row in trade_lastyr.iterrows():
    
    if row['export_country']=='United Kingdom':
        e_color='red'
    elif row['export_country']=='United States':
        e_color='blue'
    elif row['export_country']=='Canada':
        e_color='orange'
    else:
        e_color='grey'

    G_lastyear.add_edge(row['export_country'], row['import_country'], weight=row['value'], color=e_color)

# remove isolated vertices (if any)    
remove = [node for node,degree in G_lastyear.degree() if degree ==0]
G_lastyear.remove_nodes_from(remove)

# settings
options = {
     'node_color': 'grey',
     'node_size': 1000,
     'alpha': 1,
     'connectionstyle': 'arc3, rad=0.1'}  #set the connection style

colors = nx.get_edge_attributes(G_lastyear,'color').values()

weights = nx.get_edge_attributes(G_lastyear,'weight').values()
weights = np.divide(list(weights),5000)  #adjust weights so that line widths are appropriate

### Load back latest year's network graph

In [ ]:
#load back latest year's network graph
import pickle

f = open("graph_pickle_latest", "rb")
G_latest = pickle.load(f)
f.close()

### Draw graphs to compare output from last year with latest year using a 3x2 subplot containing all the network representations

In [ ]:
fig, ax = plt.subplots(3, 2, figsize = (10, 15))
layouts = {"spring": nx.spring_layout, "random": nx.random_layout, "circular": nx.circular_layout}
graphs = {"latest": G_latest, "last year's": G_lastyear}

for i, (layout_name, layout) in enumerate(layouts.items()):  
    for j, (graph_name, graph) in enumerate(graphs.items()):
        pos = layout(graph)
        nx.draw_networkx(graph, pos=pos, ax=ax[i][j], font_size=9, width=weights, edge_color=colors, **options)
        plt.tight_layout()
        ax[i][j].set_axis_off()
        ax[i][j].set_title(f"{layout_name} layout for {graph_name} data")
        
# Reference -- https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html
#   draw_networkx() -- draws graph with Matplotlib, includes options for node positions, labeling, titles, etc.
#   draw() -- simple drawing without labels or axes

In [ ]:
# What does enumerate do?
#print(list(enumerate(layouts.items()))) #enumerate adds a counter to an iterable
#print(list(enumerate(graphs.items())))